In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import cv2
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class data():
    def __init__(self, seq_length, batch_size, height, width):
        np.random.seed(100)
        self.seq_length = seq_length
        self.acc_num = 334
        self.no_acc_num = 392
        self.split = 0.95
        self.train_index = 0 
        self.train_batch_size = batch_size
        self.valid_index = 0
        self.valid_batch_size = 10
        self.height = height
        self.width = width
        self.read_annotation()
        self.shuffle_data()
        
    def shuffle_data(self):
        #img_range = np.arange(0,300-self.seq_length-60,60)
        acc_list = np.arange(1,self.acc_num+1)
        no_acc_list = np.arange(1,self.no_acc_num+1)
        list1 = np.array(np.meshgrid(1,acc_list)).T.reshape(-1,2)
        list2 = np.array(np.meshgrid(0,no_acc_list)).T.reshape(-1,2)
        shuffle_list = np.concatenate([list1, list2], axis=0)
        np.random.shuffle(shuffle_list)
        self.train = shuffle_list[:int(shuffle_list.shape[0]*self.split)]
        self.valid = shuffle_list[int(shuffle_list.shape[0]*self.split):] 
        
    def read_annotation(self):
        annotation_file = '/media/user/Hard_Disk/Dataset/child_accident_2/annotation/accident_frame.txt'
        w = open(annotation_file, "r")
        ann = w.read()
        annotation_data = []
        for i in ann.split("\n"):
            b = i.split(" ")
            if (len(b) > 1):
                annotation_data.append(b[1])
        self.annotation = np.array(annotation_data).astype("int32")
        
    def read_data(self, is_accident, dir_index):
        data = []
        label = []
        image_path = '/media/user/Hard_Disk/Dataset/child_accident_2/clip image/'
        if (is_accident):
            acc_dir = "accident/"
            range_start = self.annotation[dir_index-1] - self.seq_length
            range_end = self.annotation[dir_index-1]
            label.append([0,1])
        else:
            acc_dir = "no_accident/"
            range_start = 0
            range_end = self.seq_length
            label.append([1,0])
        dir_name = "%04d"%dir_index
        for j in range(range_start, range_end):
#             print(self.path+ acc_dir + dir_name +"/"+ str(j)+".jpg")
            img = cv2.imread(image_path + acc_dir + dir_name +"/"+ str(j)+".jpg")
#             print(img.shape)
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgRGB = cv2.resize(imgRGB, (self.height, self.width))
            data.append(imgRGB)
        
        return np.array(data), np.array(label)
        
    def next_batch(self, mode="train"):
        batch_img = []
        batch_y = []
        if (mode == "valid"):
            batch_size = self.valid_batch_size
        elif (mode == "train"):
            batch_size = self.train_batch_size
        for i in range(batch_size):
            if (mode == "train"):
                img, label = self.read_data(self.train[self.train_index+i][0], self.train[self.train_index+i][1])
            elif (mode == "valid"):
                img, label = self.read_data(self.valid[self.valid_index+i][0], self.valid[self.valid_index+i][1])
            batch_img.append(img)
            batch_y.append(label)
        if (mode == "valid"):
            self.valid_index += self.valid_batch_size
        elif (mode == "train"):
            self.train_index += self.train_batch_size
        return np.array(batch_img), np.squeeze(np.array(batch_y))
    
    def has_next(self, mode="train"):
        if (mode == "train"):
            if (self.train_index + self.train_batch_size >= self.train.shape[0]):
                return False
        elif (mode == "valid"):
            if (self.valid_index + self.valid_batch_size >= self.valid.shape[0]):
                return False
        return True
    def display_shape(self):
        print("train shape:",self.train.shape, " valid shape:",self.valid.shape)
        
    def reset_batch(self, mode="train"):
        if (mode == "train"):
            self.train_index = 0
            np.random.shuffle(self.train)
        elif (mode == "valid"):
            self.valid_index = 0
        
   

In [3]:
# Recurrent neural network (many-to-one)
class C3D(nn.Module):
    def __init__(self, num_classes):
        super(C3D, self).__init__()
        self.conv1 = nn.Conv3d(3, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv3a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv4a = nn.Conv3d(256, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv4b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv5a = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv5b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))

        self.fc6 = nn.Linear(512*1*3*3, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, num_classes)

        self.dropout = nn.Dropout(p=0.5)

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        h = self.relu(self.conv1(x))
        h = self.pool1(h)

        h = self.relu(self.conv2(h))
        h = self.pool2(h)

        h = self.relu(self.conv3a(h))
        h = self.relu(self.conv3b(h))
        h = self.pool3(h)

        h = self.relu(self.conv4a(h))
        h = self.relu(self.conv4b(h))
        h = self.pool4(h)

        h = self.relu(self.conv5a(h))
        h = self.relu(self.conv5b(h))
        h = self.pool5(h)
#         print(h.shape)
        h = h.view(-1, 512*1*3*3)
        h = self.relu(self.fc6(h))
        h = self.dropout(h)
        h = self.relu(self.fc7(h))
        h = self.dropout(h)

        logits = self.fc8(h)
        probs = self.softmax(logits)
        
#         out = self.fc2(out[:, -1, :])
        return probs

In [4]:
model = C3D(num_classes=2).to(device)

In [5]:
w = torch.from_numpy(np.array([1,1])).type(torch.FloatTensor).to(device)
criterion = nn.CrossEntropyLoss(weight=w)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [6]:
sequence_length = 30
num_epochs = 20
height = 64
width = 64
train_data = data(seq_length=sequence_length, batch_size=5, height=height, width=width)
train_data.display_shape()

train shape: (689, 2)  valid shape: (37, 2)


In [7]:
def evaluation(model):
    i = 0
    acc = 0
    while(train_data.has_next("valid")):
        img, labels = train_data.next_batch("valid")
        labels = torch.from_numpy(labels).to(device)
        img = torch.from_numpy(img.reshape(-1, 3, sequence_length, height, width)).type(torch.float).to(device)
        
        outputs = model(img)
        outputs = F.softmax(outputs)
        predict = torch.max(outputs, 1)[1]
        target = torch.max(labels, 1)[1]
        loss = criterion(outputs, target)
        correct = (predict == target).squeeze()
        acc += torch.nonzero(correct).size(0) / predict.shape[0]
        i += 1
    train_data.reset_batch("valid")
    print("validation accuracy:",acc/i)

In [8]:

# Loss and optimizer

# Train the model

for epoch in range(100):
    i = 0
    acc = 0
    print("epoch:",epoch)
    while (train_data.has_next("train")):
        img, labels = train_data.next_batch("train")
        labels = torch.from_numpy(labels).to(device)
        
        img = torch.from_numpy(img.reshape(-1, 3, sequence_length, height, width)).type(torch.float).to(device)
        
#         print(img.type())
        outputs = model(img)
        outputs = F.softmax(outputs)
#         print(outputs.shape)
#         outputs, torch.max(labels, 1)[1]
        
        predict = torch.max(outputs, 1)[1]
        target = torch.max(labels, 1)[1]
        loss = criterion(outputs, target)
        
        
        correct = (predict == target).squeeze()
#         print(predict)
#         print(target)
#         print("================")
#         print(correct)
        acc += torch.nonzero(correct).size(0) / predict.shape[0]
        
    
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        if (i+1) % 10 == 0:
            
            print ('Epoch [{}/{}], Step {}, Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, loss.item()))
            print ('train accuracy:',acc/i)
            evaluation(model)
            print("====================================")
    if (epoch % 10 == 0):
        path = "pytorch - 08/model-"+str(epoch)+".ckpt"
        torch.save(model, path)
    train_data.reset_batch("train")
# # Test the model
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images.reshape(-1, sequence_length, input_size).to(device)
#         labels = labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# # Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')

epoch: 0


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/20], Step 10, Loss: 0.8582
train accuracy: 0.4666666666666667


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


validation accuracy: 0.5333333333333333
Epoch [1/20], Step 20, Loss: 0.5810
train accuracy: 0.5263157894736841
validation accuracy: 0.5333333333333333
Epoch [1/20], Step 30, Loss: 0.7658
train accuracy: 0.503448275862069
validation accuracy: 0.5333333333333333
Epoch [1/20], Step 40, Loss: 0.9507
train accuracy: 0.5333333333333334
validation accuracy: 0.5333333333333333
Epoch [1/20], Step 50, Loss: 0.5810
train accuracy: 0.5428571428571429
validation accuracy: 0.5333333333333333
Epoch [1/20], Step 60, Loss: 0.8582
train accuracy: 0.5559322033898307
validation accuracy: 0.5333333333333333
Epoch [1/20], Step 70, Loss: 0.8582
train accuracy: 0.5217391304347828
validation accuracy: 0.5333333333333333
Epoch [1/20], Step 80, Loss: 0.7658
train accuracy: 0.5240506329113924
validation accuracy: 0.5333333333333333
Epoch [1/20], Step 90, Loss: 0.7658
train accuracy: 0.5348314606741572
validation accuracy: 0.5333333333333333
Epoch [1/20], Step 100, Loss: 0.8582
train accuracy: 0.5474747474747473
v

...

Epoch [82/20], Step 60, Loss: 0.6734
train accuracy: 0.5016949152542373
validation accuracy: 0.5333333333333333
Epoch [82/20], Step 70, Loss: 0.4885
train accuracy: 0.5130434782608695
validation accuracy: 0.5333333333333333
Epoch [82/20], Step 80, Loss: 0.6734
train accuracy: 0.5063291139240507
validation accuracy: 0.5333333333333333
Epoch [82/20], Step 90, Loss: 0.7658
train accuracy: 0.5123595505617976
validation accuracy: 0.5333333333333333
Epoch [82/20], Step 100, Loss: 0.6734
train accuracy: 0.5212121212121211
validation accuracy: 0.5333333333333333
Epoch [82/20], Step 110, Loss: 0.4885
train accuracy: 0.5357798165137613
validation accuracy: 0.5333333333333333
Epoch [82/20], Step 120, Loss: 0.6734
train accuracy: 0.5411764705882351
validation accuracy: 0.5333333333333333
Epoch [82/20], Step 130, Loss: 0.6734
train accuracy: 0.5472868217054261
validation accuracy: 0.5333333333333333
epoch: 82
Epoch [83/20], Step 10, Loss: 0.7658
train accuracy: 0.4666666666666667
validation accurac

Epoch [86/20], Step 90, Loss: 0.7658
train accuracy: 0.5303370786516857
validation accuracy: 0.5333333333333333
Epoch [86/20], Step 100, Loss: 0.6734
train accuracy: 0.5373737373737376
validation accuracy: 0.5333333333333333
Epoch [86/20], Step 110, Loss: 0.6734
train accuracy: 0.5376146788990828
validation accuracy: 0.5333333333333333
Epoch [86/20], Step 120, Loss: 0.5810
train accuracy: 0.5378151260504203
validation accuracy: 0.5333333333333333
Epoch [86/20], Step 130, Loss: 0.6734
train accuracy: 0.5302325581395351
validation accuracy: 0.5333333333333333
epoch: 86
Epoch [87/20], Step 10, Loss: 0.5810
train accuracy: 0.5555555555555556
validation accuracy: 0.5333333333333333
Epoch [87/20], Step 20, Loss: 0.5810
train accuracy: 0.6000000000000002
validation accuracy: 0.5333333333333333
Epoch [87/20], Step 30, Loss: 0.4885
train accuracy: 0.5724137931034483
validation accuracy: 0.5333333333333333
Epoch [87/20], Step 40, Loss: 0.5810
train accuracy: 0.5743589743589745
validation accurac

Epoch [90/20], Step 120, Loss: 0.8582
train accuracy: 0.5411764705882355
validation accuracy: 0.5333333333333333
Epoch [90/20], Step 130, Loss: 0.6734
train accuracy: 0.544186046511628
validation accuracy: 0.5333333333333333
epoch: 90
Epoch [91/20], Step 10, Loss: 0.7658
train accuracy: 0.42222222222222217
validation accuracy: 0.5333333333333333
Epoch [91/20], Step 20, Loss: 0.7658
train accuracy: 0.4736842105263157
validation accuracy: 0.5333333333333333
Epoch [91/20], Step 30, Loss: 0.7658
train accuracy: 0.5310344827586206
validation accuracy: 0.5333333333333333
Epoch [91/20], Step 40, Loss: 0.8582
train accuracy: 0.5333333333333332
validation accuracy: 0.5333333333333333
Epoch [91/20], Step 50, Loss: 0.5810
train accuracy: 0.5673469387755102
validation accuracy: 0.5333333333333333
Epoch [91/20], Step 60, Loss: 0.8582
train accuracy: 0.5491525423728815
validation accuracy: 0.5333333333333333
Epoch [91/20], Step 70, Loss: 0.4885
train accuracy: 0.5565217391304349
validation accuracy:

Epoch [95/20], Step 20, Loss: 0.5810
train accuracy: 0.5894736842105264
validation accuracy: 0.5333333333333333
Epoch [95/20], Step 30, Loss: 0.5810
train accuracy: 0.5655172413793104
validation accuracy: 0.5333333333333333
Epoch [95/20], Step 40, Loss: 0.5810
train accuracy: 0.5435897435897434
validation accuracy: 0.5333333333333333
Epoch [95/20], Step 50, Loss: 0.5810
train accuracy: 0.546938775510204
validation accuracy: 0.5333333333333333
Epoch [95/20], Step 60, Loss: 0.6734
train accuracy: 0.5661016949152542
validation accuracy: 0.5333333333333333
Epoch [95/20], Step 70, Loss: 0.7658
train accuracy: 0.5652173913043477
validation accuracy: 0.5333333333333333
Epoch [95/20], Step 80, Loss: 0.7658
train accuracy: 0.5518987341772151
validation accuracy: 0.5333333333333333
Epoch [95/20], Step 90, Loss: 0.6734
train accuracy: 0.5483146067415728
validation accuracy: 0.5333333333333333
Epoch [95/20], Step 100, Loss: 0.5810
train accuracy: 0.5515151515151513
validation accuracy: 0.533333333

Epoch [99/20], Step 50, Loss: 0.6734
train accuracy: 0.579591836734694
validation accuracy: 0.5333333333333333
Epoch [99/20], Step 60, Loss: 0.7658
train accuracy: 0.545762711864407
validation accuracy: 0.5333333333333333
Epoch [99/20], Step 70, Loss: 0.6734
train accuracy: 0.5333333333333335
validation accuracy: 0.5333333333333333
Epoch [99/20], Step 80, Loss: 0.7658
train accuracy: 0.5367088607594938
validation accuracy: 0.5333333333333333
Epoch [99/20], Step 90, Loss: 0.6734
train accuracy: 0.5438202247191011
validation accuracy: 0.5333333333333333
Epoch [99/20], Step 100, Loss: 0.6734
train accuracy: 0.5393939393939394
validation accuracy: 0.5333333333333333
Epoch [99/20], Step 110, Loss: 0.4885
train accuracy: 0.5394495412844037
validation accuracy: 0.5333333333333333
Epoch [99/20], Step 120, Loss: 0.4885
train accuracy: 0.5445378151260505
validation accuracy: 0.5333333333333333
Epoch [99/20], Step 130, Loss: 0.5810
train accuracy: 0.5379844961240312
validation accuracy: 0.5333333